In [2]:
import pandas as pd
from IPython.display import display

from xcai.basics import *
from xcai.plot import *

import matplotlib.pyplot as plt

from IPython.display import display,HTML

comet_ml is installed but `COMET_API_KEY` is not set.


### Helper function

In [2]:
set_plot_defaults()

In [55]:
def metrics(a,b,c,x,y):
    return {'P@1':a, 'P@5':b,'N@5':c, 'PSP@1':x, 'PSP@5':y}

def display_results(metric, index, remove_prefix=False, scale=None):
    if remove_prefix or scale:
        for m in metric:
            keys = list(m.keys())
            for k in keys:
                nk = k.split('_')[1] if remove_prefix else k
                m[nk] = m[k] if scale is None else m[k] * scale
                if nk != k: del m[k]
                
    with pd.option_context('display.precision', 4, 'display.max_columns', None):
        display(pd.DataFrame(metric, index=index))

### `Teacher`

In [7]:
# wikiseealsotitles
m = {'test_loss': 0.016681302338838577, 'test_P@1': 0.47479367940737405, 'test_P@10': 0.14173393797699302, 'test_P@3': 0.30295280211078307, 'test_P@5': 0.22635495591925775, 'test_N@1': 0.4747936725616455, 'test_N@10': 0.5081887245178223, 'test_N@3': 0.46679285168647766, 'test_N@5': 0.48181095719337463, 'test_PSP@1': 0.3659764403986233, 'test_PSP@10': 0.47102664459277543, 'test_PSP@3': 0.38466570585689525, 'test_PSP@5': 0.4126957923700924, 'test_PSN@1': 0.3659764528274536, 'test_PSN@10': 0.4537607729434967, 'test_PSN@3': 0.3976207375526428, 'test_PSN@5': 0.42237570881843567, 'test_R@200': 0.7736651615675022, 'test_R@10': 0.5737313838358737, 'test_R@100': 0.7436145963127184, 'test_runtime': 239.3287, 'test_samples_per_second': 741.721, 'test_steps_per_second': 0.464}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,47.4794,30.2953,22.6355,14.1734,47.4794,46.6793,48.1811,50.8189,36.5976,38.4666,41.2696,47.1027,36.5976,39.7621,42.2376,45.3761,57.3731,74.3615,77.3665,0.0167,239.3287,741.721,0.464


In [10]:
# wikititles
m = {'test_loss': 0.017709823325276375, 'test_P@1': 0.6604039844693987, 'test_P@10': 0.1922625146260841, 'test_P@3': 0.4427169790774948, 'test_P@5': 0.3219657464253877, 'test_N@1': 0.6604039669036865, 'test_N@10': 0.5267409682273865, 'test_N@3': 0.5628888010978699, 'test_N@5': 0.534044623374939, 'test_PSP@1': 0.38866437063552794, 'test_PSP@10': 0.45216630925636925, 'test_PSP@3': 0.41840363402702435, 'test_PSP@5': 0.4250382267401933, 'test_PSN@1': 0.3886643648147583, 'test_PSN@10': 0.47987130284309387, 'test_PSN@3': 0.42873772978782654, 'test_PSN@5': 0.4506883919239044, 'test_R@200': 0.702557668379058, 'test_R@10': 0.510692117939397, 'test_R@100': 0.663280306014536, 'test_runtime': 580.8063, 'test_samples_per_second': 1349.405, 'test_steps_per_second': 0.844}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,66.0404,44.2717,32.1966,19.2263,66.0404,56.2889,53.4045,52.6741,38.8664,41.8404,42.5038,45.2166,38.8664,42.8738,45.0688,47.9871,51.0692,66.328,70.2558,0.0177,580.8063,1349.405,0.844


In [7]:
# wikiseealso
m = {'test_loss': 0.015018783509731293, 'test_P@1': 0.5037095456721967, 'test_P@10': 0.15165478973596774, 'test_P@3': 0.3256964200210398, 'test_P@5': 0.2433811227220156, 'test_N@1': 0.5037095546722412, 'test_N@10': 0.541210949420929, 'test_N@3': 0.4979328513145447, 'test_N@5': 0.5138446688652039, 'test_PSP@1': 0.3672674882608596, 'test_PSP@10': 0.49842471202063104, 'test_PSP@3': 0.4018981757157289, 'test_PSP@5': 0.43512711505254204, 'test_PSN@1': 0.3672674894332886, 'test_PSN@10': 0.474685400724411, 'test_PSN@3': 0.41203731298446655, 'test_PSN@5': 0.4404144287109375, 'test_R@200': 0.8056290164783148, 'test_R@10': 0.6113826946844456, 'test_R@100': 0.7781915615151795, 'test_runtime': 316.9593, 'test_samples_per_second': 560.056, 'test_steps_per_second': 0.35}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,50.371,32.5696,24.3381,15.1655,50.371,49.7933,51.3845,54.1211,36.7267,40.1898,43.5127,49.8425,36.7267,41.2037,44.0414,47.4685,61.1383,77.8192,80.5629,0.015,316.9593,560.056,0.35


In [8]:
# wikipedia
m = {'test_loss': 0.010197877883911133, 'test_P@1': 0.8240430855522792, 'test_P@10': 0.3066724678888317, 'test_P@3': 0.6387183043424678, 'test_P@5': 0.4950069091535431, 'test_N@1': 0.8240430951118469, 'test_N@10': 0.7470307946205139, 'test_N@3': 0.7715214490890503, 'test_N@5': 0.750892162322998, 'test_PSP@1': 0.4637808932625801, 'test_PSP@10': 0.6563743396782328, 'test_PSP@3': 0.5579214660003528, 'test_PSP@5': 0.5994651830178395, 'test_PSN@1': 0.46378087997436523, 'test_PSN@10': 0.6357831954956055, 'test_PSN@3': 0.5512413382530212, 'test_PSN@5': 0.5932626128196716, 'test_R@200': 0.8997065010083698, 'test_R@10': 0.7399693088722892, 'test_R@100': 0.8772790846554102, 'test_runtime': 658.7976, 'test_samples_per_second': 1189.657, 'test_steps_per_second': 0.372}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,82.4043,63.8718,49.5007,30.6672,82.4043,77.1521,75.0892,74.7031,46.3781,55.7921,59.9465,65.6374,46.3781,55.1241,59.3263,63.5783,73.9969,87.7279,89.9707,0.0102,658.7976,1189.657,0.372


### `momos M1`

In [9]:
# wikiseealsotitles
m = {'test_loss': 0.20433445274829865, 'test_P@1': 0.3429512998901501, 'test_P@10': 0.11108300706984563, 'test_P@3': 0.23351829422867218, 'test_P@5': 0.17679632707113338, 'test_N@1': 0.34295129776000977, 'test_N@10': 0.37233859300613403, 'test_N@3': 0.3428209722042084, 'test_N@5': 0.35341933369636536, 'test_PSP@1': 0.27993850226241385, 'test_PSP@10': 0.376466313139454, 'test_PSP@3': 0.30754761672224956, 'test_PSP@5': 0.3316568403739362, 'test_PSN@1': 0.2799384891986847, 'test_PSN@10': 0.34672781825065613, 'test_PSN@3': 0.30699291825294495, 'test_PSN@5': 0.3245813846588135, 'test_R@200': 0.5835736040171667, 'test_R@10': 0.4208163969078128, 'test_R@100': 0.5554816765045649, 'test_runtime': 280.87, 'test_samples_per_second': 632.018, 'test_steps_per_second': 0.395}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,34.2951,23.3518,17.6796,11.1083,34.2951,34.2821,35.3419,37.2339,27.9939,30.7548,33.1657,37.6466,27.9938,30.6993,32.4581,34.6728,42.0816,55.5482,58.3574,0.2043,280.87,632.018,0.395


In [94]:
#wikiseealsotitles

# test_enc + 0.2 * test_teacher_lbl
m = {'P@1': 0.34457933132411345, 'P@10': 0.11202489930430193, 'P@3': 0.23517261452085578, 'P@5': 0.17794777906110715, 'N@1': 0.34457934, 'N@10': 0.37385765, 'N@3': 0.3442961, 'N@5': 0.35469761, 'PSP@1': 0.27901799771375035, 'PSP@10': 0.3792134699643732, 'PSP@3': 0.30876249855090515, 'PSP@5': 0.33322251235980027, 'PSN@1': 0.27901798, 'PSN@10': 0.34732234, 'PSN@3': 0.30716676, 'PSN@5': 0.32484227, 'R@200': 0.5852598297168966, 'R@10': 0.42251354813475966, 'R@100': 0.561321716537376}
display_metric(m, remove_prefix=False)

# test_enc + 0.3 * test_centroid_repr
m = {'P@1': 0.3472889614962116, 'P@10': 0.11220178576460135, 'P@3': 0.23585424705895192, 'P@5': 0.178364645241428, 'N@1': 0.34728897, 'N@10': 0.37524232, 'N@3': 0.3459366, 'N@5': 0.3562202, 'PSP@1': 0.2836158698437905, 'PSP@10': 0.38013037113037834, 'PSP@3': 0.3111235493277344, 'PSP@5': 0.3349052664347519, 'PSN@1': 0.28361586, 'PSN@10': 0.3493827, 'PSN@3': 0.310036, 'PSN@5': 0.32726714, 'R@200': 0.5847146349994797, 'R@10': 0.4227361736324652, 'R@100': 0.5574637129920544}
display_metric(m, remove_prefix=False)

# test_clf + 0.5 * test_centroid_fused_repr
m = {'P@1': 0.3486972931864913, 'P@10': 0.1130129848181999, 'P@3': 0.23705602343465887, 'P@5': 0.17949018392829996, 'N@1': 0.3486973, 'N@10': 0.3771655, 'N@3': 0.34739548, 'N@5': 0.35785893, 'PSP@1': 0.28390518763666195, 'PSP@10': 0.38173471548426585, 'PSP@3': 0.31175215012557717, 'PSP@5': 0.3360194378754333, 'PSN@1': 0.28390518, 'PSN@10': 0.350171, 'PSN@3': 0.3104189, 'PSN@5': 0.32781738, 'R@200': 0.588366797901596, 'R@10': 0.425270609770189, 'R@100': 0.5596008034348555}
display_metric(m, remove_prefix=False)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,34.4579,23.5173,17.7948,11.2025,34.4579,34.4296,35.4698,37.3858,27.9018,30.8762,33.3223,37.9213,27.9018,30.7167,32.4842,34.7322,42.2514,56.1322,58.526


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,34.7289,23.5854,17.8365,11.2202,34.7289,34.5937,35.622,37.5242,28.3616,31.1124,33.4905,38.013,28.3616,31.0036,32.7267,34.9383,42.2736,55.7464,58.4715


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,34.8697,23.7056,17.949,11.3013,34.8697,34.7395,35.7859,37.7165,28.3905,31.1752,33.6019,38.1735,28.3905,31.0419,32.7817,35.0171,42.5271,55.9601,58.8367


In [92]:
# wikititles
m = {'test_loss': 0.06967095285654068, 'test_P@1': 0.4629782977328027, 'test_P@10': 0.09752674027060981, 'test_P@3': 0.2477470293199689, 'test_P@5': 0.16989217128495665, 'test_N@1': 0.4629783034324646, 'test_N@10': 0.3309706151485443, 'test_N@3': 0.34786248207092285, 'test_N@5': 0.33060941100120544, 'test_PSP@1': 0.2689172497073804, 'test_PSP@10': 0.24868763014515857, 'test_PSP@3': 0.25220004154671294, 'test_PSP@5': 0.24359712211063567, 'test_PSN@1': 0.2689172625541687, 'test_PSN@10': 0.3140500485897064, 'test_PSN@3': 0.27733322978019714, 'test_PSN@5': 0.29243215918540955, 'test_R@200': 0.45237561830988215, 'test_R@10': 0.32226054502319607, 'test_R@100': 0.42211712448585054, 'test_runtime': 1703.2392, 'test_samples_per_second': 460.149, 'test_steps_per_second': 0.575}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,46.2978,24.7747,16.9892,9.7527,46.2978,34.7862,33.0609,33.0971,26.8917,25.22,24.3597,24.8688,26.8917,27.7333,29.2432,31.405,32.2261,42.2117,45.2376,0.0697,1703.2392,460.149,0.575


In [97]:
#wikititles

# test_enc + 0.3 * test_teacher_lbl
m = {'P@1': 0.4647033530124033, 'P@10': 0.09813382703248204, 'P@3': 0.24976724938361214, 'P@5': 0.17115023674777796, 'N@1': 0.46470335, 'N@10': 0.33239007, 'N@3': 0.34985363, 'N@5': 0.3322393, 'PSP@1': 0.266296795978827, 'PSP@10': 0.24874687392961845, 'PSP@3': 0.25235819042771834, 'PSP@5': 0.24374869640865912, 'PSN@1': 0.2662968, 'PSN@10': 0.3132064, 'PSN@3': 0.27656326, 'PSN@5': 0.29164457, 'R@200': 0.4406619692485824, 'R@10': 0.3234623150481888, 'R@100': 0.41583325165414353}
display_metric(m, remove_prefix=False)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,46.4703,24.9767,17.115,9.8134,46.4703,34.9854,33.2239,33.239,26.6297,25.2358,24.3749,24.8747,26.6297,27.6563,29.1645,31.3206,32.3462,41.5833,44.0662


In [4]:
#wikiseealso
m = {'test_loss': 0.1637713611125946, 'test_P@1': 0.48389713545334195, 'test_P@10': 0.14728445483470948, 'test_P@3': 0.31811771775172687, 'test_P@5': 0.23777934259078015, 'test_N@1': 0.4838971495628357, 'test_N@10': 0.5204978585243225, 'test_N@3': 0.48093181848526, 'test_N@5': 0.4956223964691162, 'test_PSP@1': 0.3471672765048718, 'test_PSP@10': 0.47972551736149643, 'test_PSP@3': 0.38718797803337185, 'test_PSP@5': 0.4206066509767296, 'test_PSN@1': 0.3471672534942627, 'test_PSN@10': 0.4514100253582001, 'test_PSN@3': 0.39258873462677, 'test_PSN@5': 0.4197971522808075, 'test_R@200': 0.7652431698696025, 'test_R@10': 0.5857672396317298, 'test_R@100': 0.7389251695375327, 'test_runtime': 354.4095, 'test_samples_per_second': 500.875, 'test_steps_per_second': 0.313}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,48.3897,31.8118,23.7779,14.7284,48.3897,48.0932,49.5622,52.0498,34.7167,38.7188,42.0607,47.9726,34.7167,39.2589,41.9797,45.141,58.5767,73.8925,76.5243,0.1638,354.4095,500.875,0.313


In [9]:
#wikiseealso

# test_enc + 0.9 * test_teacher_lbl
m = {'P@1': 0.4926175252795538, 'P@10': 0.15095175055618268, 'P@3': 0.32434629937394543, 'P@5': 0.24306340309268976, 'N@1': 0.49261752, 'N@10': 0.5301417, 'N@3': 0.48901075, 'N@5': 0.5043292, 'PSP@1': 0.34847975871404174, 'PSP@10': 0.4925119161157266, 'PSP@3': 0.39387665692628143, 'PSP@5': 0.4300987601327263, 'PSN@1': 0.34847975, 'PSN@10': 0.45926732, 'PSN@3': 0.39760143, 'PSN@5': 0.42626452, 'R@200': 0.7865898066958945, 'R@10': 0.5977345456137035, 'R@100': 0.7590797989587106}
display_metric(m, remove_prefix=False)

# test_enc + 0.6 * test_centroid_repr
m = {'P@1': 0.4930625580936822, 'P@10': 0.14932766245095766, 'P@3': 0.32266005689677707, 'P@5': 0.24090020561642803, 'N@1': 0.49306256, 'N@10': 0.52717984, 'N@3': 0.48790085, 'N@5': 0.50213796, 'PSP@1': 0.36139826033163214, 'PSP@10': 0.48929007965248594, 'PSP@3': 0.39825073226521923, 'PSP@5': 0.4301212071581012, 'PSN@1': 0.36139825, 'PSN@10': 0.4610039, 'PSN@3': 0.4035537, 'PSN@5': 0.42961562, 'R@200': 0.7682362092089037, 'R@10': 0.5909517316098264, 'R@100': 0.7460220673711099}
display_metric(m, remove_prefix=False)

# test_clf + 0.9 * test_centroid_fused_repr
m = {'P@1': 0.4962341210601921, 'P@10': 0.15043742782289565, 'P@3': 0.3246880545307812, 'P@5': 0.24256879700306508, 'N@1': 0.49623412, 'N@10': 0.53012913, 'N@3': 0.4905924, 'N@5': 0.5049278, 'PSP@1': 0.3615449989425466, 'PSP@10': 0.49166717282959804, 'PSP@3': 0.39935205078505587, 'PSP@5': 0.4317239936704363, 'PSN@1': 0.36154497, 'PSN@10': 0.46223307, 'PSN@3': 0.40426677, 'PSN@5': 0.4305397, 'R@200': 0.7869761880866443, 'R@10': 0.5941071112220179, 'R@100': 0.7501463400002143}
display_metric(m, remove_prefix=False)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,49.2618,32.4346,24.3063,15.0952,49.2618,48.9011,50.4329,53.0142,34.848,39.3877,43.0099,49.2512,34.848,39.7601,42.6265,45.9267,59.7735,75.908,78.659


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,49.3063,32.266,24.09,14.9328,49.3063,48.7901,50.2138,52.718,36.1398,39.8251,43.0121,48.929,36.1398,40.3554,42.9616,46.1004,59.0952,74.6022,76.8236


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,49.6234,32.4688,24.2569,15.0437,49.6234,49.0592,50.4928,53.0129,36.1545,39.9352,43.1724,49.1667,36.1545,40.4267,43.054,46.2233,59.4107,75.0146,78.6976


In [5]:
#wikipedia

### `momos M2`

In [9]:
#wikiseealsotitles
m = {'test_loss': 0.08250453323125839, 'test_P@1': 0.3504098245218714, 'test_P@10': 0.11164014308651794, 'test_P@3': 0.2374109230206094, 'test_P@5': 0.178951637889937, 'test_N@1': 0.3504098355770111, 'test_N@10': 0.3743511736392975, 'test_N@3': 0.3474583923816681, 'test_N@5': 0.3568243980407715, 'test_PSP@1': 0.2752314455371148, 'test_PSP@10': 0.371782579869313, 'test_PSP@3': 0.3047629916581384, 'test_PSP@5': 0.328367531139476, 'test_PSN@1': 0.2752314507961273, 'test_PSN@10': 0.34207022190093994, 'test_PSN@3': 0.3036549389362335, 'test_PSN@5': 0.320774644613266, 'test_R@200': 0.5670612413135152, 'test_R@10': 0.41908470502431966, 'test_R@100': 0.5416234576180196, 'test_runtime': 55.8045, 'test_samples_per_second': 3181.014, 'test_steps_per_second': 1.989}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,35.041,23.7411,17.8952,11.164,35.041,34.7458,35.6824,37.4351,27.5231,30.4763,32.8368,37.1783,27.5231,30.3655,32.0775,34.207,41.9085,54.1623,56.7061,0.0825,55.8045,3181.014,1.989


In [18]:
#wikiseealsotitles

# test_clf + 0.2 * test_enc
m = {'P@1': 0.35120412359518915, 'P@10': 0.11225980903024012, 'P@3': 0.23841365518408894, 'P@5': 0.17970988367198423, 'N@1': 0.35120413, 'N@10': 0.37621465, 'N@3': 0.34886736, 'N@5': 0.35838717, 'PSP@1': 0.2775433379150663, 'PSP@10': 0.3749818434087103, 'PSP@3': 0.3073157950437174, 'PSP@5': 0.33099236118375347, 'PSN@1': 0.27754334, 'PSN@10': 0.344846, 'PSN@3': 0.30608308, 'PSN@5': 0.32330644, 'R@200': 0.568667339855646, 'R@10': 0.4216408496040447, 'R@100': 0.5477761420753614} 
display_metric(m, remove_prefix=False)

# test_clf + 0.2 * test_centroid_repr
m = {'P@1': 0.3512829901698448, 'P@10': 0.11263329859450111, 'P@3': 0.23842116628643664, 'P@5': 0.18003098329736691, 'N@1': 0.35128298, 'N@10': 0.3764666, 'N@3': 0.34867075, 'N@5': 0.35832953, 'PSP@1': 0.27860606061826354, 'PSP@10': 0.3763109585337449, 'PSP@3': 0.3079967554608065, 'PSP@5': 0.3319881798798843, 'PSN@1': 0.27860606, 'PSN@10': 0.3453594, 'PSN@3': 0.30648392, 'PSN@5': 0.3236756, 'R@200': 0.5683758520722021, 'R@10': 0.4221968231113175, 'R@100': 0.5490152025392615}
display_metric(m, remove_prefix=False)

# test_clf + 0.2 * test_centroid_fused_repr
m = {'P@1': 0.3517336563107343, 'P@10': 0.11297862152495697, 'P@3': 0.23898449896254897, 'P@5': 0.18057628932784378, 'N@1': 0.35173365, 'N@10': 0.37736115, 'N@3': 0.3494572, 'N@5': 0.35922197, 'PSP@1': 0.2781064653233376, 'PSP@10': 0.37673358347444735, 'PSP@3': 0.30804218106809594, 'PSP@5': 0.3322534324269027, 'PSN@1': 0.27810645, 'PSN@10': 0.3454442, 'PSN@3': 0.30646953, 'PSN@5': 0.32375866, 'R@200': 0.5687609045426391, 'R@10': 0.423278324106155, 'R@100': 0.5505767445182107}
display_metric(m, remove_prefix=False)

# test_clf + 0.2 * test_teacher_lbl
m = {'P@1': 0.35111962369377236, 'P@10': 0.11297749485960493, 'P@3': 0.23933564299732582, 'P@5': 0.18057966932390176, 'N@1': 0.35111964, 'N@10': 0.37655032, 'N@3': 0.3491049, 'N@5': 0.35857683, 'PSP@1': 0.27534147796991787, 'PSP@10': 0.3768198420841615, 'PSP@3': 0.3075191100491229, 'PSP@5': 0.3317501507879783, 'PSN@1': 0.27534145, 'PSN@10': 0.34427264, 'PSN@3': 0.30513218, 'PSN@5': 0.3224494, 'R@200': 0.5691823668902801, 'R@10': 0.42222068634594395, 'R@100': 0.5499031959716303}
display_metric(m, remove_prefix=False)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,35.1204,23.8414,17.971,11.226,35.1204,34.8867,35.8387,37.6215,27.7543,30.7316,33.0992,37.4982,27.7543,30.6083,32.3306,34.4846,42.1641,54.7776,56.8667


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,35.1283,23.8421,18.0031,11.2633,35.1283,34.8671,35.833,37.6467,27.8606,30.7997,33.1988,37.6311,27.8606,30.6484,32.3676,34.5359,42.2197,54.9015,56.8376


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,35.1734,23.8984,18.0576,11.2979,35.1734,34.9457,35.9222,37.7361,27.8106,30.8042,33.2253,37.6734,27.8106,30.647,32.3759,34.5444,42.3278,55.0577,56.8761


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,35.112,23.9336,18.058,11.2977,35.112,34.9105,35.8577,37.655,27.5341,30.7519,33.175,37.682,27.5341,30.5132,32.2449,34.4273,42.2221,54.9903,56.9182


In [10]:
#wikititles

In [88]:
#wikiseealso
m = {'test_loss': 0.07303251326084137, 'test_P@1': 0.4962341210601921, 'test_P@10': 0.14843816015538108, 'test_P@3': 0.3241265996302599, 'test_P@5': 0.24121116525364142, 'test_N@1': 0.49623411893844604, 'test_N@10': 0.5264792442321777, 'test_N@3': 0.4897010028362274, 'test_N@5': 0.5030137300491333, 'test_PSP@1': 0.35037947101621003, 'test_PSP@10': 0.47986042047680927, 'test_PSP@3': 0.39040447606380496, 'test_PSP@5': 0.4229050246933454, 'test_PSN@1': 0.35037946701049805, 'test_PSN@10': 0.4533686637878418, 'test_PSN@3': 0.3960227370262146, 'test_PSN@5': 0.42271676659584045, 'test_R@200': 0.7599179831449293, 'test_R@10': 0.5878175209915208, 'test_R@100': 0.7346757515330188, 'test_runtime': 54.4109, 'test_samples_per_second': 3262.492, 'test_steps_per_second': 2.04}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,49.6234,32.4127,24.1211,14.8438,49.6234,48.9701,50.3014,52.6479,35.0379,39.0404,42.2905,47.986,35.0379,39.6023,42.2717,45.3369,58.7818,73.4676,75.9918,0.073,54.4109,3262.492,2.04


In [89]:
#wikiseealsotitles

# test_clf + 0.01 * test_enc
m = {'P@1': 0.49622285440666986, 'P@10': 0.1484668901218626, 'P@3': 0.324182932897871, 'P@5': 0.24123144522998158, 'N@1': 0.49622285, 'N@10': 0.5265218, 'N@3': 0.48976392, 'N@5': 0.50304073, 'PSP@1': 0.3504044787480384, 'PSP@10': 0.47999861343997924, 'PSP@3': 0.39054336453686045, 'PSP@5': 0.4229904347886971, 'PSN@1': 0.35040447, 'PSN@10': 0.45343733, 'PSN@3': 0.39612716, 'PSN@5': 0.42277542, 'R@200': 0.7611193054986555, 'R@10': 0.5878917561818745, 'R@100': 0.7348906556887708}
display_metric(m, remove_prefix=False)

# test_clf + 0.4 * test_centroid_repr
m = {'P@1': 0.49800861898994453, 'P@10': 0.1499957750048236, 'P@3': 0.32588607535531405, 'P@5': 0.2428876432977423, 'N@1': 0.4980086, 'N@10': 0.53013414, 'N@3': 0.49223953, 'N@5': 0.50588125, 'PSP@1': 0.3592605305165714, 'PSP@10': 0.48824481867201525, 'PSP@3': 0.3980289192474755, 'PSP@5': 0.43009552436457266, 'PSN@1': 0.3592605, 'PSN@10': 0.46049184, 'PSN@3': 0.4033273, 'PSN@5': 0.42950034, 'R@200': 0.7617607183969056, 'R@10': 0.5922950438608355, 'R@100': 0.7421327264074206}
display_metric(m, remove_prefix=False)

# test_clf + 0.5 * test_centroid_fused_repr
m = {'P@1': 0.4996253837703856, 'P@10': 0.150869503985472, 'P@3': 0.3272061515930053, 'P@5': 0.24401430864996002, 'N@1': 0.49962538, 'N@10': 0.53217065, 'N@3': 0.493847, 'N@5': 0.5076394, 'PSP@1': 0.3586697051007238, 'PSP@10': 0.4904074897823853, 'PSP@3': 0.3985794714022731, 'PSP@5': 0.43125056202381223, 'PSN@1': 0.3586697, 'PSN@10': 0.46134338, 'PSN@3': 0.40348014, 'PSN@5': 0.42999536, 'R@200': 0.7641193904341601, 'R@10': 0.594970435240746, 'R@100': 0.7433394776094998}
display_metric(m, remove_prefix=False)

# test_clf + 0.3 * test_teacher_lbl
m = {'P@1': 0.49858321831957864, 'P@10': 0.15091964059364482, 'P@3': 0.3272286849000526, 'P@5': 0.2442723150156196, 'N@1': 0.49858323, 'N@10': 0.5319181, 'N@3': 0.4932509, 'N@5': 0.5072888, 'PSP@1': 0.3510656951548298, 'PSP@10': 0.4892410225302072, 'PSP@3': 0.3946054190724195, 'PSP@5': 0.4292972734866941, 'PSN@1': 0.3510657, 'PSN@10': 0.45876184, 'PSN@3': 0.39908114, 'PSN@5': 0.42678356, 'R@200': 0.762169280692369, 'R@10': 0.5961890524301934, 'R@100': 0.745169439552785}
display_metric(m, remove_prefix=False)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,49.6223,32.4183,24.1231,14.8467,49.6223,48.9764,50.3041,52.6522,35.0404,39.0543,42.299,47.9999,35.0404,39.6127,42.2775,45.3437,58.7892,73.4891,76.1119


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,49.8009,32.5886,24.2888,14.9996,49.8009,49.224,50.5881,53.0134,35.9261,39.8029,43.0096,48.8245,35.926,40.3327,42.95,46.0492,59.2295,74.2133,76.1761


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,49.9625,32.7206,24.4014,15.087,49.9625,49.3847,50.7639,53.2171,35.867,39.8579,43.1251,49.0407,35.867,40.348,42.9995,46.1343,59.497,74.3339,76.4119


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,49.8583,32.7229,24.4272,15.092,49.8583,49.3251,50.7289,53.1918,35.1066,39.4605,42.9297,48.9241,35.1066,39.9081,42.6784,45.8762,59.6189,74.5169,76.2169


In [11]:
#wikipedia

### `momos oracle M1`

momos trained with ground truth categories.

In [74]:
# wikiseealso
m = {'test_loss': 0.2057356834411621, 'test_P@1': 0.335735008309157, 'test_P@10': 0.11074050080277051, 'test_P@3': 0.23009510933349214, 'test_P@5': 0.17522462890478116, 'test_N@1': 0.335735023021698, 'test_N@10': 0.3687324821949005, 'test_N@3': 0.33729374408721924, 'test_N@5': 0.34893161058425903, 'test_PSP@1': 0.26585926910254837, 'test_PSP@10': 0.37290697863291716, 'test_PSP@3': 0.2984864886953208, 'test_PSP@5': 0.32521100461832175, 'test_PSN@1': 0.2658592760562897, 'test_PSN@10': 0.3385559320449829, 'test_PSN@3': 0.29634883999824524, 'test_PSN@5': 0.3152778744697571, 'test_R@200': 0.585841842472852, 'test_R@10': 0.42049191781195305, 'test_R@100': 0.5585277892872079, 'test_runtime': 248.9278, 'test_samples_per_second': 713.118, 'test_steps_per_second': 0.446}
display_metric(m)

metrics = [{'N@1': 0.4250852167606354, 'N@10': 0.45932623744010925, 'N@3': 0.4246317744255066, 'N@5': 0.43779298663139343, 'P@1': 0.4250852040672619, 'P@10': 0.13018899811279538, 'P@3': 0.27998385113005086, 'P@5': 0.20988874179657227, 'PSN@1': 0.32125014066696167, 'PSN@10': 0.4048474133014679, 'PSN@3': 0.35667988657951355, 'PSN@5': 0.37882766127586365, 'PSP@1': 0.3212501436007495, 'PSP@10': 0.4290657480890762, 'PSP@3': 0.3516265864155488, 'PSP@5': 0.37920318389758756, 'R@10': 0.5154495558094366, 'R@100': 0.6618621904056633, 'R@200': 0.690253706619285},
           {'P@1': 0.335735008309157, 'P@10': 0.11074050080277051, 'P@3': 0.23009510933349214, 'P@5': 0.17522462890478116, 'N@1': 0.335735023021698, 'N@10': 0.3687324821949005, 'N@3': 0.33729374408721924, 'N@5': 0.34893161058425903, 'PSP@1': 0.26585926910254837, 'PSP@10': 0.37290697863291716, 'PSP@3': 0.2984864886953208, 'PSP@5': 0.32521100461832175, 'PSN@1': 0.2658592760562897, 'PSN@10': 0.3385559320449829, 'PSN@3': 0.29634883999824524, 'PSN@5': 0.3152778744697571, 'R@200': 0.585841842472852, 'R@10': 0.42049191781195305, 'R@100': 0.5585277892872079}]

display_results(metrics, index=['Ground truth', 'Linker'], scale=100)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,33.5735,23.0095,17.5225,11.0741,33.5735,33.7294,34.8932,36.8732,26.5859,29.8486,32.5211,37.2907,26.5859,29.6349,31.5278,33.8556,42.0492,55.8528,58.5842,0.2057,248.9278,713.118,0.446


,N@1,N@10,N@3,N@5,P@1,P@10,P@3,P@5,PSN@1,PSN@10,PSN@3,PSN@5,PSP@1,PSP@10,PSP@3,PSP@5,R@10,R@100,R@200
Ground truth,42.5085,45.9326,42.4632,43.7793,42.5085,13.0189,27.9984,20.9889,32.1250,40.4847,35.6680,37.8828,32.1250,42.9066,35.1627,37.9203,51.5450,66.1862,69.0254
Linker,33.5735,36.8732,33.7294,34.8932,33.5735,11.0741,23.0095,17.5225,26.5859,33.8556,29.6349,31.5278,26.5859,37.2907,29.8486,32.5211,42.0492,55.8528,58.5842


#### `Inference with ground truth`

Metrics with ground truth metadata.

In [57]:
# wikiseealsotitles
m = {'test_loss': 0.19825103878974915, 'test_P@1': 0.37074050080274906, 'test_P@10': 0.1188265780356551, 'test_P@3': 0.25063421870457464, 'test_P@5': 0.18944990564195896, 'test_N@1': 0.37074050307273865, 'test_N@10': 0.4057261645793915, 'test_N@3': 0.3724217712879181, 'test_N@5': 0.3848222494125366, 'test_PSP@1': 0.2962928678575879, 'test_PSP@10': 0.3982671959567921, 'test_PSP@3': 0.32501365704165813, 'test_PSP@5': 0.35058293049541067, 'test_PSN@1': 0.29629287123680115, 'test_PSN@10': 0.37048059701919556, 'test_PSN@3': 0.3266501724720001, 'test_PSN@5': 0.3462205231189728, 'test_R@200': 0.6369637154078921, 'test_R@10': 0.4608445217062401, 'test_R@100': 0.6069939450201214, 'test_runtime': 245.6463, 'test_samples_per_second': 722.645, 'test_steps_per_second': 0.452}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,37.0741,25.0634,18.945,11.8827,37.0741,37.2422,38.4822,40.5726,29.6293,32.5014,35.0583,39.8267,29.6293,32.665,34.6221,37.0481,46.0845,60.6994,63.6964,0.1983,245.6463,722.645,0.452


In [62]:
# wikiseealsotitles
# number of categories: 5
m = {'test_loss': 0.19786709547042847, 'test_P@1': 0.3702391347210095, 'test_P@10': 0.11890882460636706, 'test_P@3': 0.25066050756279223, 'test_P@5': 0.18955468551971572, 'test_N@1': 0.37023913860321045, 'test_N@10': 0.405764102935791, 'test_N@3': 0.3722630441188812, 'test_N@5': 0.38483309745788574, 'test_PSP@1': 0.2960730216869164, 'test_PSP@10': 0.398552853430692, 'test_PSP@3': 0.3251863600330204, 'test_PSP@5': 0.3508256123083663, 'test_PSN@1': 0.2960730195045471, 'test_PSN@10': 0.37052610516548157, 'test_PSN@3': 0.32662031054496765, 'test_PSN@5': 0.3462768495082855, 'test_R@200': 0.6386829313483762, 'test_R@10': 0.46123652095134443, 'test_R@100': 0.6081462389523458, 'test_runtime': 295.7903, 'test_samples_per_second': 600.138, 'test_steps_per_second': 0.375}
o = display_metric(m)

# number of categories: 10
m = {'test_loss': 0.19777467846870422, 'test_P@1': 0.3701490014928316, 'test_P@10': 0.11889079796073136, 'test_P@3': 0.2505403299252217, 'test_P@5': 0.1895377855394318, 'test_N@1': 0.37014901638031006, 'test_N@10': 0.4056752026081085, 'test_N@3': 0.37211939692497253, 'test_N@5': 0.38474544882774353, 'test_PSP@1': 0.2961520500099008, 'test_PSP@10': 0.39854476945783035, 'test_PSP@3': 0.3251778360242046, 'test_PSP@5': 0.35086973315236486, 'test_PSN@1': 0.2961520552635193, 'test_PSN@10': 0.3705241084098816, 'test_PSN@3': 0.3266165852546692, 'test_PSN@5': 0.346291184425354, 'test_R@200': 0.639163230813345, 'test_R@10': 0.46118741409794906, 'test_R@100': 0.6085032436962176, 'test_runtime': 390.59, 'test_samples_per_second': 454.479, 'test_steps_per_second': 0.284}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,37.0239,25.0661,18.9555,11.8909,37.0239,37.2263,38.4833,40.5764,29.6073,32.5186,35.0826,39.8553,29.6073,32.662,34.6277,37.0526,46.1237,60.8146,63.8683,0.1979,295.7903,600.138,0.375


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,37.0149,25.054,18.9538,11.8891,37.0149,37.2119,38.4745,40.5675,29.6152,32.5178,35.087,39.8545,29.6152,32.6617,34.6291,37.0524,46.1187,60.8503,63.9163,0.1978,390.59,454.479,0.284


In [68]:
# wikititles
m = {'test_loss': 0.06927984207868576, 'test_P@1': 0.45808128429855194, 'test_P@10': 0.09496620703453507, 'test_P@3': 0.24056737986854726, 'test_P@5': 0.16484255680662488, 'test_N@1': 0.45808127522468567, 'test_N@10': 0.32611966133117676, 'test_N@3': 0.34089207649230957, 'test_N@5': 0.32468974590301514, 'test_PSP@1': 0.2692115310141104, 'test_PSP@10': 0.24277318787610716, 'test_PSP@3': 0.24363384608297156, 'test_PSP@5': 0.23566574430676646, 'test_PSN@1': 0.2692115306854248, 'test_PSN@10': 0.31049057841300964, 'test_PSN@3': 0.27161550521850586, 'test_PSN@5': 0.28750190138816833, 'test_R@200': 0.4766446025879506, 'test_R@10': 0.3208551847210097, 'test_R@100': 0.4399206996658419, 'test_runtime': 1554.1354, 'test_samples_per_second': 504.295, 'test_steps_per_second': 0.631}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,45.8081,24.0567,16.4843,9.4966,45.8081,34.0892,32.469,32.612,26.9212,24.3634,23.5666,24.2773,26.9212,27.1616,28.7502,31.0491,32.0855,43.9921,47.6645,0.0693,1554.1354,504.295,0.631


In [77]:
# wikititles
# number of categories: 5
m = {'test_loss': 0.07321329414844513, 'test_P@1': 0.4580889398693194, 'test_P@10': 0.0949674829629965, 'test_P@3': 0.24056057491675414, 'test_P@5': 0.16484026013539468, 'test_N@1': 0.4580889344215393, 'test_N@10': 0.3261234760284424, 'test_N@3': 0.3408890962600708, 'test_N@5': 0.3246893286705017, 'test_PSP@1': 0.2692090893675554, 'test_PSP@10': 0.2427803836190167, 'test_PSP@3': 0.2436241110461253, 'test_PSP@5': 0.2356614507681842, 'test_PSN@1': 0.269209086894989, 'test_PSN@10': 0.3104959726333618, 'test_PSN@3': 0.2716104984283447, 'test_PSN@5': 0.28750085830688477, 'test_R@200': 0.476652661222211, 'test_R@10': 0.3208593249395337, 'test_R@100': 0.439929277887563, 'test_runtime': 1074.2185, 'test_samples_per_second': 729.594, 'test_steps_per_second': 0.456}
o = display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,45.8089,24.0561,16.484,9.4967,45.8089,34.0889,32.4689,32.6123,26.9209,24.3624,23.5661,24.278,26.9209,27.161,28.7501,31.0496,32.0859,43.9929,47.6653,0.0732,1074.2185,729.594,0.456


### Results

#### Teacher

In [12]:
models = ['NGAME (teacher)', 'RAMEN (baseline)']
datasets = ['LF-WikiSeeAlsoTitles-320K', 'LF-WikiTitles-500K', 'LF-WikiSeeAlso-320K', 'LF-Wikipedia-500K']

index = pd.MultiIndex.from_tuples([(d, m) for d in datasets for m in models], names=["Dataset", "Model"])

metric = [metrics(47.4794, 22.6355, 48.1811, 36.5976, 41.2696), metrics(35.2, 18.4, 36.5, 29.0,  34.5),
          metrics(66.0404, 32.1966, 53.4045, 38.8664, 42.5038), metrics(46.1, 17.4, 33.8, 30.5, 25.7),
          metrics(50.573, 24.322, 51.423, 36.779, 43.385), metrics(47.29, 23.37, 49.15, 33.81, 41.95),
          metrics(82.54, 48.759, 74.495, 46.612, 59.492), metrics(81.07, 50.1, 75.27, 50.94, 61.84)
         ]

display_results(metric, index)

P@1    P@5    N@5  PSP@1  PSP@5
Dataset                   Model                                              
LF-WikiSeeAlsoTitles-320K NGAME (teacher)   47.48  22.64  48.18  36.60  41.27
                          RAMEN (baseline)  35.20  18.40  36.50  29.00  34.50
LF-WikiTitles-500K        NGAME (teacher)   66.04  32.20  53.40  38.87  42.50
                          RAMEN (baseline)  46.10  17.40  33.80  30.50  25.70
LF-WikiSeeAlso-320K       NGAME (teacher)   50.57  24.32  51.42  36.78  43.38
                          RAMEN (baseline)  47.29  23.37  49.15  33.81  41.95
LF-Wikipedia-500K         NGAME (teacher)   82.54  48.76  74.50  46.61  59.49
                          RAMEN (baseline)  81.07  50.10  75.27  50.94  61.84

#### LF-WikiSeeAlsoTitles-320K

In [10]:
dataset_name = 'LF-WikiSeeAlsoTitles-320K'
models = ['teacher', 'momos-oracle', 'oak-oracle', 'momos', 'oak']

index = pd.MultiIndex.from_tuples([(dataset_name, o) for o in models], names=["Dataset", "Model"])

metric = [metrics(47.4794, 22.6355, 48.1811, 36.5976, 41.2696),
          metrics(40.92, 19.97, 40.889, 30.1417, 35.3729),
          metrics(38.92, 19.35, 40.35, 29.69, 34.85),
          metrics(34.87, 17.95, 35.86, 28.39, 33.6),
          metrics(32.75, 16.64, 33.65, 25.67, 30.51)]

display_results(metric, index)

P@1    P@5    N@5  PSP@1  PSP@5
Dataset                   Model                                          
LF-WikiSeeAlsoTitles-320K teacher       47.48  22.64  48.18  36.60  41.27
                          momos-oracle  40.92  19.97  40.89  30.14  35.37
                          oak-oracle    38.92  19.35  40.35  29.69  34.85
                          momos         34.87  17.95  35.86  28.39  33.60
                          oak           32.75  16.64  33.65  25.67  30.51

#### Summary

In [21]:
dataset_name = ['LF-WikiSeeAlsoTitles-320K', 'LF-WikiTitles-500K', 'LF-WikiSeeAlso-320K']
models = ['teacher', 'momos', 'oak']

ll = [(d, o) for d in dataset_name for o in models]
ll[-1] = (ll[-1][0], '*'+ll[-1][1])

index = pd.MultiIndex.from_tuples(ll, names=["Dataset", "Model"])

metric = [
    metrics(47.48, 22.64, 48.18, 36.60, 41.27),
    metrics(34.87, 17.95, 35.86, 28.39, 33.60),
    metrics(32.75, 16.64, 33.65, 25.67, 30.51),
    
    metrics(66.0404, 32.1966, 53.4045, 38.8664, 42.5038),
    metrics(46.25, 16.91, 32.98, '-', '-'),
    metrics(43.80, 17.23, 32.94, 26.00, 24.42),

    metrics(50.37, 24.34, 51.38, 36.73, 43.51),
    metrics(49.62, 24.25, 50.49, 36.15, 43.17),
    metrics(48.57, 23.28, 49.16,33.92, 40.44),
]

display_results(metric, index)

P@1    P@5    N@5  PSP@1  PSP@5
Dataset                   Model                                     
LF-WikiSeeAlsoTitles-320K teacher  47.48  22.64  48.18   36.6  41.27
                          momos    34.87  17.95  35.86  28.39   33.6
                          oak      32.75  16.64  33.65  25.67  30.51
LF-WikiTitles-500K        teacher  66.04  32.20  53.40  38.87   42.5
                          momos    46.25  16.91  32.98      -      -
                          oak      43.80  17.23  32.94   26.0  24.42
LF-WikiSeeAlso-320K       teacher  50.37  24.34  51.38  36.73  43.51
                          momos    49.62  24.25  50.49  36.15  43.17
                          *oak     48.57  23.28  49.16  33.92  40.44

### Ablation

#### `NGAME`

In [22]:
# distillation with NGAME
index = pd.MultiIndex.from_tuples([('LF-WikiSeeAlsoTitles-320K', 'momos'),('LF-WikiSeeAlsoTitles-320K','NGAME')], names=["Dataset", "Model"])

metric = [metrics(34.87, 17.95, 35.86, 28.39, 33.60),
          metrics(32.58, 16.63, 33.58, 27.12, 31.62)]

display_results(metric, index)

P@1    P@5    N@5  PSP@1  PSP@5
Dataset                   Model                                   
LF-WikiSeeAlsoTitles-320K momos  34.87  17.95  35.86  28.39  33.60
                          NGAME  32.58  16.63  33.58  27.12  31.62

#### `Linker`

In [83]:
# New linker trained with distillation
# 23_ngame-linker-for-wikiseealsotitles-005
m = {'test_P@1': 0.465070557417683, 'test_P@10': 0.12219868743486595, 'test_P@3': 0.27601423353900517, 'test_P@5': 0.200201673098182, 'test_N@1': 0.46507054567337036, 'test_N@10': 0.3666885495185852, 'test_N@3': 0.36969348788261414, 'test_N@5': 0.3578966557979584, 'test_PSP@1': 0.3376232234413037, 'test_PSP@10': 0.3061427324367076, 'test_PSP@3': 0.29390382410647337, 'test_PSP@5': 0.2891282799804326, 'test_PSN@1': 0.3376232087612152, 'test_PSN@10': 0.35979506373405457, 'test_PSN@3': 0.32121309638023376, 'test_PSN@5': 0.33437737822532654, 'test_R@200': 0.5627873233700605, 'test_R@10': 0.37060635198234976, 'test_R@100': 0.5219418347935953, 'test_runtime': 83.5619, 'test_samples_per_second': 2124.353, 'test_steps_per_second': 1.328}
display_metric(m)

# Comparision
metrics = [{'P@1': 0.46157789482578937, 'P@10': 0.129043742782284, 'P@3': 0.2856228112179042, 'P@5': 0.2097163619976852, 'N@1': 0.4615779, 'N@10': 0.37416404, 'N@3': 0.37562373, 'N@5': 0.3647661, 'PSP@1': 0.2667712445772757, 'PSP@10': 0.2646211921257092, 'PSP@3': 0.2467218111928937, 'PSP@5': 0.24707461387146157, 'PSN@1': 0.26677126, 'PSN@10': 0.30799657, 'PSN@3': 0.2677922, 'PSN@5': 0.28322187, 'R@200': 0.43547917964633065, 'R@10': 0.3834404785658011, 'R@100': 0.43547917964633065},
           {'P@1': 0.465070557417683, 'P@10': 0.12219868743486595, 'P@3': 0.27601423353900517, 'P@5': 0.200201673098182, 'N@1': 0.46507054567337036, 'N@10': 0.3666885495185852, 'N@3': 0.36969348788261414, 'N@5': 0.3578966557979584, 'PSP@1': 0.3376232234413037, 'PSP@10': 0.3061427324367076, 'PSP@3': 0.29390382410647337, 'PSP@5': 0.2891282799804326, 'PSN@1': 0.3376232087612152, 'PSN@10': 0.35979506373405457, 'PSN@3': 0.32121309638023376, 'PSN@5': 0.33437737822532654, 'R@200': 0.5627873233700605, 'R@10': 0.37060635198234976, 'R@100': 0.5219418347935953}]

display_results(metrics, index=['OAK Linker', 'Our Linker'], scale=100)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,runtime,samples_per_second,steps_per_second
0,46.5071,27.6014,20.0202,12.2199,46.5071,36.9693,35.7897,36.6689,33.7623,29.3904,28.9128,30.6143,33.7623,32.1213,33.4377,35.9795,37.0606,52.1942,56.2787,83.5619,2124.353,1.328


,P@1,P@10,P@3,P@5,N@1,N@10,N@3,N@5,PSP@1,PSP@10,PSP@3,PSP@5,PSN@1,PSN@10,PSN@3,PSN@5,R@200,R@10,R@100
OAK Linker,46.1578,12.9044,28.5623,20.9716,46.1578,37.4164,37.5624,36.4766,26.6771,26.4621,24.6722,24.7075,26.6771,30.7997,26.7792,28.3222,43.5479,38.3440,43.5479
Our Linker,46.5071,12.2199,27.6014,20.0202,46.5071,36.6689,36.9693,35.7897,33.7623,30.6143,29.3904,28.9128,33.7623,35.9795,32.1213,33.4377,56.2787,37.0606,52.1942


In [86]:
# New linker trained with distillation
# 23_ngame-linker-for-wikiseealsotitles-005
m = {'test_P@1': 0.8641964442879775, 'test_P@10': 0.2623216300529071, 'test_P@3': 0.6015093933100634, 'test_P@5': 0.44198031401867777, 'test_N@1': 0.8641964197158813, 'test_N@10': 0.7603024244308472, 'test_N@3': 0.7839914560317993, 'test_N@5': 0.7592983245849609, 'test_PSP@1': 0.5963366050986504, 'test_PSP@10': 0.6010842175233736, 'test_PSP@3': 0.5967820320477192, 'test_PSP@5': 0.5879113702207276, 'test_PSN@1': 0.5963366031646729, 'test_PSN@10': 0.6915479302406311, 'test_PSN@3': 0.6282311081886292, 'test_PSN@5': 0.6523230075836182, 'test_R@200': 0.8800618741080505, 'test_R@10': 0.7467741136365903, 'test_R@100': 0.8608030019322284, 'test_runtime': 220.1044, 'test_samples_per_second': 3148.879, 'test_steps_per_second': 1.972}
display_metric(m)

# Comparision
metrics = [{'P@1': 0.889415682415645, 'P@10': 0.36619317771985593, 'P@3': 0.7237820825439242, 'P@5': 0.5785275046826103, 'N@1': 0.8894157, 'N@10': 0.853537, 'N@3': 0.86265427, 'N@5': 0.85278, 'PSP@1': 0.655130884446039, 'PSP@10': 0.8594903419789359, 'PSP@3': 0.7732849090613231, 'PSP@5': 0.8153301442406827, 'PSN@1': 0.65513086, 'PSN@10': 0.82323587, 'PSN@3': 0.75117874, 'PSN@5': 0.7888541, 'R@200': 0.8932376452161794, 'R@10': 0.8446453160626738, 'R@100': 0.8932376452161794},
           {'P@1': 0.8641964442879775, 'P@10': 0.2623216300529071, 'P@3': 0.6015093933100634, 'P@5': 0.44198031401867777, 'N@1': 0.8641964197158813, 'N@10': 0.7603024244308472, 'N@3': 0.7839914560317993, 'N@5': 0.7592983245849609, 'PSP@1': 0.5963366050986504, 'PSP@10': 0.6010842175233736, 'PSP@3': 0.5967820320477192, 'PSP@5': 0.5879113702207276, 'PSN@1': 0.5963366031646729, 'PSN@10': 0.6915479302406311, 'PSN@3': 0.6282311081886292, 'PSN@5': 0.6523230075836182, 'R@200': 0.8800618741080505, 'R@10': 0.7467741136365903, 'R@100': 0.8608030019322284}]
           
display_results(metrics, index=['OAK Linker', 'Our Linker'], scale=100)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,runtime,samples_per_second,steps_per_second
0,86.4196,60.1509,44.198,26.2322,86.4196,78.3991,75.9298,76.0302,59.6337,59.6782,58.7911,60.1084,59.6337,62.8231,65.2323,69.1548,74.6774,86.0803,88.0062,220.1044,3148.879,1.972


,P@1,P@10,P@3,P@5,N@1,N@10,N@3,N@5,PSP@1,PSP@10,PSP@3,PSP@5,PSN@1,PSN@10,PSN@3,PSN@5,R@200,R@10,R@100
OAK Linker,88.9416,36.6193,72.3782,57.8528,88.9416,85.3537,86.2654,85.2780,65.5131,85.9490,77.3285,81.5330,65.5131,82.3236,75.1179,78.8854,89.3238,84.4645,89.3238
Our Linker,86.4196,26.2322,60.1509,44.1980,86.4196,76.0302,78.3991,75.9298,59.6337,60.1084,59.6782,58.7911,59.6337,69.1548,62.8231,65.2323,88.0062,74.6774,86.0803


In [87]:
# inference of MOMOS trained with the old renee linker
metrics = [{'test_loss': 0.2043614387512207, 'test_P@1': 0.34220206743092135, 'test_P@10': 0.1110582204320963, 'test_P@3': 0.23252682871871455, 'test_P@5': 0.17639861420179906, 'test_N@1': 0.3422020673751831, 'test_N@10': 0.3715559244155884, 'test_N@3': 0.3414316773414612, 'test_N@5': 0.3523217737674713, 'test_PSP@1': 0.27896923450071315, 'test_PSP@10': 0.37695292566712124, 'test_PSP@3': 0.30651840978857203, 'test_PSP@5': 0.3313276872945231, 'test_PSN@1': 0.2789692282676697, 'test_PSN@10': 0.3463619649410248, 'test_PSN@3': 0.3059273362159729, 'test_PSN@5': 0.32387575507164, 'test_R@200': 0.5836970283971985, 'test_R@10': 0.4203938894730459, 'test_R@100': 0.5562429263402626, 'test_runtime': 260.3641, 'test_samples_per_second': 681.795, 'test_steps_per_second': 0.426},
           {'test_loss': 0.20433445274829865, 'test_P@1': 0.3429512998901501, 'test_P@10': 0.11108300706984563, 'test_P@3': 0.23351829422867218, 'test_P@5': 0.17679632707113338, 'test_N@1': 0.34295129776000977, 'test_N@10': 0.37233859300613403, 'test_N@3': 0.3428209722042084, 'test_N@5': 0.35341933369636536, 'test_PSP@1': 0.27993850226241385, 'test_PSP@10': 0.376466313139454, 'test_PSP@3': 0.30754761672224956, 'test_PSP@5': 0.3316568403739362, 'test_PSN@1': 0.2799384891986847, 'test_PSN@10': 0.34672781825065613, 'test_PSN@3': 0.30699291825294495, 'test_PSN@5': 0.3245813846588135, 'test_R@200': 0.5835736040171667, 'test_R@10': 0.4208163969078128, 'test_R@100': 0.5554816765045649, 'test_runtime': 280.87, 'test_samples_per_second': 632.018, 'test_steps_per_second': 0.395}]
        
display_results(metrics, index=['MOMOS new linker', 'MOMOS renee linker'], scale=100, remove_prefix=True)

,loss,P@1,P@10,P@3,P@5,N@1,N@10,N@3,N@5,PSP@1,PSP@10,PSP@3,PSP@5,PSN@1,PSN@10,PSN@3,PSN@5,R@200,R@10,R@100,runtime,samples,steps
MOMOS new linker,20.4361,34.2202,11.1058,23.2527,17.6399,34.2202,37.1556,34.1432,35.2322,27.8969,37.6953,30.6518,33.1328,27.8969,34.6362,30.5927,32.3876,58.3697,42.0394,55.6243,26036.41,68179.5,42.6
MOMOS renee linker,20.4334,34.2951,11.1083,23.3518,17.6796,34.2951,37.2339,34.2821,35.3419,27.9939,37.6466,30.7548,33.1657,27.9938,34.6728,30.6993,32.4581,58.3574,42.0816,55.5482,28087.00,63201.8,39.5


#### `MOMOS`

In [7]:
# MOMOS trained with label free parameters
m = {'test_loss': 0.14033186435699463, 'test_P@1': 0.33785877249809876, 'test_P@10': 0.11063177759628172, 'test_P@3': 0.23049507553353324, 'test_P@5': 0.17498464918475493, 'test_N@1': 0.33785876631736755, 'test_N@10': 0.3685618042945862, 'test_N@3': 0.33806565403938293, 'test_N@5': 0.3490257263183594, 'test_PSP@1': 0.26296250709866, 'test_PSP@10': 0.3705992357286006, 'test_PSP@3': 0.2953603554953699, 'test_PSP@5': 0.3217639742594924, 'test_PSN@1': 0.2629624903202057, 'test_PSN@10': 0.33637887239456177, 'test_PSN@3': 0.2940034568309784, 'test_PSN@5': 0.31280088424682617, 'test_R@200': 0.5801263512240644, 'test_R@10': 0.41852711871054366, 'test_R@100': 0.5538106731905881, 'test_runtime': 260.2416, 'test_samples_per_second': 682.116, 'test_steps_per_second': 0.427}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,33.7859,23.0495,17.4985,11.0632,33.7859,33.8066,34.9026,36.8562,26.2963,29.536,32.1764,37.0599,26.2962,29.4003,31.2801,33.6379,41.8527,55.3811,58.0126,0.1403,260.2416,682.116,0.427


In [59]:
# MOMOS trained with only the distillation loss
m = {'test_loss': 0.244585320353508, 'test_P@1': 0.32695828521533393, 'test_P@10': 0.10685125200690773, 'test_P@3': 0.22268352909143516, 'test_P@5': 0.16922513590417926, 'test_N@1': 0.3269582986831665, 'test_N@10': 0.35452887415885925, 'test_N@3': 0.3256007134914398, 'test_N@5': 0.3360194265842438, 'test_PSP@1': 0.26029174301672003, 'test_PSP@10': 0.35887802357311965, 'test_PSP@3': 0.2878276998261999, 'test_PSP@5': 0.3130290098640511, 'test_PSN@1': 0.2602917551994324, 'test_PSN@10': 0.3260513246059418, 'test_PSN@3': 0.28627878427505493, 'test_PSN@5': 0.3039938509464264, 'test_R@200': 0.5496826728803664, 'test_R@10': 0.40189315629719036, 'test_R@100': 0.525948614535817, 'test_runtime': 246.8328, 'test_samples_per_second': 719.171, 'test_steps_per_second': 0.45}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,32.6958,22.2684,16.9225,10.6851,32.6958,32.5601,33.6019,35.4529,26.0292,28.7828,31.3029,35.8878,26.0292,28.6279,30.3994,32.6051,40.1893,52.5949,54.9683,0.2446,246.8328,719.171,0.45


In [78]:
# MOMOS with curriculum learning
# for first 10 epochs model trained on ground truth
# 10-50 epochs with mixture of ground truth and linker
# after 50 solely on linker predictions
m = {'eval_N@1': 0.34260767698287964, 'eval_N@10': 0.3710184395313263, 'eval_N@3': 0.34144872426986694, 'eval_N@5': 0.352095365524292, 'eval_P@1': 0.3426076669577219, 'eval_P@10': 0.11071627749769722, 'eval_P@3': 0.23240477330555334, 'eval_P@5': 0.17606512125753881, 'eval_PSN@1': 0.2780739665031433, 'eval_PSN@10': 0.3449312448501587, 'eval_PSN@3': 0.3048427104949951, 'eval_PSN@5': 0.32255592942237854, 'eval_PSP@1': 0.27807398600919436, 'eval_PSP@10': 0.3753249551006196, 'eval_PSP@3': 0.30533764541164254, 'eval_PSP@5': 0.3297364349546069, 'eval_R@10': 0.41913106370655095, 'eval_R@100': 0.5558299901356762, 'eval_R@200': 0.5835159286261691, 'eval_loss': 0.14285016059875488, 'eval_runtime': 239.7765, 'eval_samples_per_second': 740.335, 'eval_steps_per_second': 0.463}
display_metric(m)

,N@1,N@3,N@5,N@10,P@1,P@3,P@5,P@10,PSN@1,PSN@3,PSN@5,PSN@10,PSP@1,PSP@3,PSP@5,PSP@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,34.2608,34.1449,35.2095,37.1018,34.2608,23.2405,17.6065,11.0716,27.8074,30.4843,32.2556,34.4931,27.8074,30.5338,32.9736,37.5325,41.9131,55.583,58.3516,0.1429,239.7765,740.335,0.463


In [10]:
# MOMOS with the new linker
m = {'eval_N@1': 0.3428161144256592, 'eval_N@10': 0.3726022243499756, 'eval_N@3': 0.34249943494796753, 'eval_N@5': 0.35309654474258423, 'eval_P@1': 0.34281610004788327, 'eval_P@10': 0.11154268653355172, 'eval_P@3': 0.23369292735826286, 'eval_P@5': 0.17681773371282458, 'eval_PSN@1': 0.27551478147506714, 'eval_PSN@10': 0.34465792775154114, 'eval_PSN@3': 0.30383044481277466, 'eval_PSN@5': 0.3216642439365387, 'eval_PSP@1': 0.27551480179072646, 'eval_PSP@10': 0.3771481141172554, 'eval_PSP@3': 0.3055325028787715, 'eval_PSP@5': 0.32996276678899294, 'eval_R@10': 0.42188439026324975, 'eval_R@100': 0.5586695699229649, 'eval_R@200': 0.5863339563836947, 'eval_loss': 0.14264144003391266}
display_metric(m)

,N@1,N@3,N@5,N@10,P@1,P@3,P@5,P@10,PSN@1,PSN@3,PSN@5,PSN@10,PSP@1,PSP@3,PSP@5,PSP@10,R@10,R@100,R@200,loss
0,34.2816,34.2499,35.3097,37.2602,34.2816,23.3693,17.6818,11.1543,27.5515,30.383,32.1664,34.4658,27.5515,30.5533,32.9963,37.7148,42.1884,55.867,58.6334,0.1426


__Teacher as source of memory__

In [4]:
# Metadata datapoint centroid
m = {'eval_N@1': 0.34199362993240356, 'eval_N@10': 0.37102705240249634, 'eval_N@3': 0.3409167230129242, 'eval_N@5': 0.3518246114253998, 'eval_P@1': 0.3419936343407599, 'eval_P@10': 0.11089541728870185, 'eval_P@3': 0.23217005135717475, 'eval_P@5': 0.1760369546237333, 'eval_PSN@1': 0.27753040194511414, 'eval_PSN@10': 0.3450949192047119, 'eval_PSN@3': 0.30440929532051086, 'eval_PSN@5': 0.32249584794044495, 'eval_PSP@1': 0.2775304170472095, 'eval_PSP@10': 0.37618305400264973, 'eval_PSP@3': 0.3051767104600429, 'eval_PSP@5': 0.33008956225833397, 'eval_R@10': 0.4196254879158511, 'eval_R@100': 0.555012793775215, 'eval_R@200': 0.5827565720718949, 'eval_loss': 0.14277608692646027}
display_metric(m)

,N@1,N@3,N@5,N@10,P@1,P@3,P@5,P@10,PSN@1,PSN@3,PSN@5,PSN@10,PSP@1,PSP@3,PSP@5,PSP@10,R@10,R@100,R@200,loss
0,34.1994,34.0917,35.1825,37.1027,34.1994,23.217,17.6037,11.0895,27.753,30.4409,32.2496,34.5095,27.753,30.5177,33.009,37.6183,41.9625,55.5013,58.2757,0.1428


In [5]:
# Metadata label centroid
m = {'eval_N@1': 0.33611243963241577, 'eval_N@10': 0.3675803244113922, 'eval_N@3': 0.33693501353263855, 'eval_N@5': 0.34841859340667725, 'eval_P@1': 0.33611244120215195, 'eval_P@10': 0.11025490803596373, 'eval_P@3': 0.22998619834944153, 'eval_P@5': 0.17501281581855946, 'eval_PSN@1': 0.2722616493701935, 'eval_PSN@10': 0.341418594121933, 'eval_PSN@3': 0.30053114891052246, 'eval_PSN@5': 0.3189307451248169, 'eval_PSP@1': 0.2722616527246265, 'eval_PSP@10': 0.37364044182286615, 'eval_PSP@3': 0.30218431540420715, 'eval_PSP@5': 0.32775921876754316, 'eval_R@10': 0.4173007805362308, 'eval_R@100': 0.5546183358469483, 'eval_R@200': 0.5819529648440122, 'eval_loss': 0.1429023146629333}
display_metric(m)

,N@1,N@3,N@5,N@10,P@1,P@3,P@5,P@10,PSN@1,PSN@3,PSN@5,PSN@10,PSP@1,PSP@3,PSP@5,PSP@10,R@10,R@100,R@200,loss
0,33.6112,33.6935,34.8419,36.758,33.6112,22.9986,17.5013,11.0255,27.2262,30.0531,31.8931,34.1419,27.2262,30.2184,32.7759,37.364,41.7301,55.4618,58.1953,0.1429


#### `DEXA`

In [9]:
# LF-WikiSeeAlsoTitles-320K

# DEXA 
m = {'test_loss': 0.025145720690488815, 'test_P@1': 0.31571979832690195, 'test_P@10': 0.10227586401154627, 'test_P@3': 0.21283647391298272, 'test_P@5': 0.16139368504087023, 'test_N@1': 0.3157198131084442, 'test_N@10': 0.34602293372154236, 'test_N@3': 0.316232293844223, 'test_N@5': 0.32706084847450256, 'test_PSP@1': 0.2564183512109082, 'test_PSP@10': 0.34358976403171615, 'test_PSP@3': 0.2780649867496514, 'test_PSP@5': 0.2999490465863107, 'test_PSN@1': 0.2564183473587036, 'test_PSN@10': 0.3202965557575226, 'test_PSN@3': 0.2813890278339386, 'test_PSN@5': 0.29843130707740784, 'test_R@200': 0.5651670575537319, 'test_R@10': 0.3944086211906528, 'test_R@100': 0.5339327154347112, 'test_runtime': 195.3196, 'test_samples_per_second': 908.844, 'test_steps_per_second': 0.568}
display_metric(m)

# DEXA with distillation; tdsl=1.0, d_mse=0.1, sdtl=1.0, l_mse=0.1
m = {}
display_metric(m)

# DEXA with distillation; tdsl=1.0, d_mse=0.1
m = {}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,31.572,21.2836,16.1394,10.2276,31.572,31.6232,32.7061,34.6023,25.6418,27.8065,29.9949,34.359,25.6418,28.1389,29.8431,32.0297,39.4409,53.3933,56.5167,0.0251,195.3196,908.844,0.568


""
0


""
0


### Qualitative Analysis

`/data/Projects/xc_nlg/outputs/86-distillation-for-wikiseealso-with-oak-7-3-4/examples/85-oak-dr-ep-for-wikiseealso-with-additive-renee-embedding-5-3_examples.csv`

In [7]:
o = """<text style=color:red>data_input_text</text>: <text style=color:green>Grumman XP-50</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['Lockheed XP-49', 'Westland Whirlwind', 'Gloster F.9/37', 'Grumman XP-50', 'Lockheed P-38 Lightning']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['Low-wing aircraft', 'Twin-engined tractor aircraft', 'Aircraft first flown in 1941', 'United States fighter aircraft 1940â\x80\x931949', 'Grumman aircraft']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Carrier-based aircraft', 'Cancelled military aircraft projects of the United States', 'Grumman aircraft']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>Grumman XP-50</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['Curtiss XP-46', 'Lockheed XP-49', 'Grumman XP-50', 'Curtiss XP-42', 'Grumman F2F']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['Low-wing aircraft', 'Twin-engined tractor aircraft', 'Aircraft first flown in 1941', 'United States fighter aircraft 1940â\x80\x931949', 'Grumman aircraft']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Carrier-based aircraft', 'Cancelled military aircraft projects of the United States', 'Grumman aircraft']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>Grumman XP-50</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['Focke-Wulf Fw 187', 'Westland Whirlwind', 'Kawasaki Ki-45', 'Gloster F.9/37', 'IMAM Ro.57', 'Lockheed P-38 Lightning', 'Mikoyan-Gurevich DIS', 'Tairov Ta-3', 'Grumman F7F Tigercat', 'Nakajima J1N', 'PZL.38 Wilk']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['Low-wing aircraft', 'Twin-engined tractor aircraft', 'Aircraft first flown in 1941', 'United States fighter aircraft 1940â\x80\x931949', 'Grumman aircraft']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Carrier-based aircraft', 'Cancelled military aircraft projects of the United States', 'Grumman aircraft']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>Tangbe</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['Tangbe', 'Mustang District', 'Kali Gandaki River', 'Upper Mustang', 'Gandaki River']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['Populated places in Mustang District']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Populated places in Cameroon', 'Communes of Cameroon', 'Township-level divisions of Hebei']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>Tangbe</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['Tangbe', 'Desalpur', 'Vladivostok', 'Kitenge', 'List of currently erupting volcanoes']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['Populated places in Mustang District']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Populated places in Cameroon', 'Communes of Cameroon', 'Township-level divisions of Hebei']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>Tangbe</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['Mustang District', 'Kali Gandaki Gorge', 'Kali Gandaki River', 'Upper Mustang', 'Gandaki River']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['Populated places in Mustang District']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Populated places in Cameroon', 'Communes of Cameroon', 'Township-level divisions of Hebei']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>PZL-230 Skorpion</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['Atlas Carver', 'Novi Avion', 'PZL-230 Skorpion', 'Embraer MFT-LF', 'Scaled Composites ARES']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['Low-wing aircraft', 'Canard aircraft', 'Twinjets', 'PZL aircraft', 'Cancelled military aircraft projects of Poland']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Low-wing aircraft', 'Twin-engined tractor aircraft', 'PZL aircraft']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>PZL-230 Skorpion</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['CASA C-101 Aviojet', 'Fouga CM.170 Magister', 'PZL M28 Skytruck', 'PZL-106 Kruk', 'PZL-230 Skorpion']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['Low-wing aircraft', 'Canard aircraft', 'Twinjets', 'PZL aircraft', 'Cancelled military aircraft projects of Poland']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Low-wing aircraft', 'Twin-engined tractor aircraft', 'PZL aircraft']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>PZL-230 Skorpion</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['Atlas Carver', 'IAR 95', 'FMA SAIA 90', 'Novi Avion', 'Embraer MFT-LF', 'Scaled Composites ARES']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['Low-wing aircraft', 'Canard aircraft', 'Twinjets', 'PZL aircraft', 'Cancelled military aircraft projects of Poland']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Low-wing aircraft', 'Twin-engined tractor aircraft', 'PZL aircraft']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>Hearts of the World</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['The Kaiser, Beast of Berlin', 'To Hell with the Kaiser!', 'Hearts of the World', 'The Heart of Humanity', 'Over the Rhine']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['American films', 'American silent feature films', 'American black-and-white films', 'American drama films', 'Articles containing video clips', 'Paramount Pictures films', 'Films shot in London', 'Films shot in Los Angeles', 'Films shot in France', 'Films directed by D. W. Griffith', '1918 films', 'Famous Players-Lasky films', 'Films shot in England', 'American World War I propaganda films', '1910s war films']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Windows games', 'Paradox Interactive games', 'Video games developed in Sweden']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>Hearts of the World</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['Hearts of the World', 'List of grand strategy video games', 'List of Paradox Interactive games', 'Chronology of grand strategy video games', 'Hearts of Steel']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['American films', 'American silent feature films', 'American black-and-white films', 'American drama films', 'Articles containing video clips', 'Paramount Pictures films', 'Films shot in London', 'Films shot in Los Angeles', 'Films shot in France', 'Films directed by D. W. Griffith', '1918 films', 'Famous Players-Lasky films', 'Films shot in England', 'American World War I propaganda films', '1910s war films']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Windows games', 'Paradox Interactive games', 'Video games developed in Sweden']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>Hearts of the World</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['The Kaiser, Beast of Berlin', 'To Hell with the Kaiser!', 'Yankee Doodle in Berlin', 'The Heart of Humanity', 'Over the Rhine', 'The False Faces', 'The Unpardonable Sin', 'The Battle Cry of Peace', 'Civilization']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['American films', 'American silent feature films', 'American black-and-white films', 'American drama films', 'Articles containing video clips', 'Paramount Pictures films', 'Films shot in London', 'Films shot in Los Angeles', 'Films shot in France', 'Films directed by D. W. Griffith', '1918 films', 'Famous Players-Lasky films', 'Films shot in England', 'American World War I propaganda films', '1910s war films']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Windows games', 'Paradox Interactive games', 'Video games developed in Sweden']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>Venezuelan protests (2014âpresent)</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['Timeline of the 2014 Venezuelan protests', 'Timeline of the 2015 Venezuelan protests', 'Timeline of the 2017 Venezuelan protests', 'Timeline of the 2018 Venezuelan protests', 'Timeline of the 2019 Venezuelan protests']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['Articles containing video clips', 'Protests in Venezuela', 'Riots and civil disorder in Venezuela', '21st century in Caracas', '2014 protests', '2015 protests', '2014 riots', 'Venezuelan protests (2014â\x80\x93present)', '2014 in Venezuela', '2016 protests', '2017 protests', '2017 in Venezuela', 'Ongoing protests', '2016 in Venezuela', '2015 in Venezuela']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Protests in Venezuela', 'Venezuelan protests (2014â\x80\x93present)', 'Venezuela-related lists']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>Venezuelan protests (2014âpresent)</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['Bolivarian Revolution', '2014 Venezuelan protests', '2019 Venezuelan blackouts', 'Crisis in Venezuela', '2017 Venezuelan National Assembly attack']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['Articles containing video clips', 'Protests in Venezuela', 'Riots and civil disorder in Venezuela', '21st century in Caracas', '2014 protests', '2015 protests', '2014 riots', 'Venezuelan protests (2014â\x80\x93present)', '2014 in Venezuela', '2016 protests', '2017 protests', '2017 in Venezuela', 'Ongoing protests', '2016 in Venezuela', '2015 in Venezuela']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Protests in Venezuela', 'Venezuelan protests (2014â\x80\x93present)', 'Venezuela-related lists']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>Venezuelan protests (2014âpresent)</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['Timeline of the 2014 Venezuelan protests', 'Timeline of the 2015 Venezuelan protests', 'Timeline of the 2017 Venezuelan protests', 'Timeline of the 2018 Venezuelan protests', 'Timeline of the 2019 Venezuelan protests']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['Articles containing video clips', 'Protests in Venezuela', 'Riots and civil disorder in Venezuela', '21st century in Caracas', '2014 protests', '2015 protests', '2014 riots', 'Venezuelan protests (2014â\x80\x93present)', '2014 in Venezuela', '2016 protests', '2017 protests', '2017 in Venezuela', 'Ongoing protests', '2016 in Venezuela', '2015 in Venezuela']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Protests in Venezuela', 'Venezuelan protests (2014â\x80\x93present)', 'Venezuela-related lists']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>Mecca2Medina</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['Islamic music', 'DAM (band)', 'Native Deen', 'Dirty Kuffar', 'Mecca2Medina']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['Converts to Islam', 'Living people', 'English Muslims', 'Performers of Islamic music', 'Musical groups established in 1996', 'English hip hop groups', 'Iqra Promotions artists']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Egyptian architecture', 'Buildings and structures in Mecca', 'Mecca']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>Mecca2Medina</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['Mosque', 'Delicate Arch', 'Hajj', 'Mecca2Medina', 'List of ziyarat locations']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['Converts to Islam', 'Living people', 'English Muslims', 'Performers of Islamic music', 'Musical groups established in 1996', 'English hip hop groups', 'Iqra Promotions artists']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Egyptian architecture', 'Buildings and structures in Mecca', 'Mecca']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>Mecca2Medina</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['Black British', 'List of converts to Islam', 'Islamic music', 'Nasheed', 'DAM (band)', 'British hip hop', 'Native Deen', 'Dirty Kuffar']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['Converts to Islam', 'Living people', 'English Muslims', 'Performers of Islamic music', 'Musical groups established in 1996', 'English hip hop groups', 'Iqra Promotions artists']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Egyptian architecture', 'Buildings and structures in Mecca', 'Mecca']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>Sayn</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['Sayn-Wittgenstein-Berleburg', 'Sayn-Wittgenstein-Hohenstein', 'Sayn-Wittgenstein-Sayn', 'Sponheim-Sayn', 'Sayn-Homburg']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['States and territories established in the 11th century', 'History of the Rhineland', 'Former states and territories of Rhineland-Palatinate', 'History of the Westerwald']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Communes of PyrÃ©nÃ©es-Atlantiques', 'Populated places in Badakhshan Province', 'Irish-language feminine given names']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>Sayn</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['Balkh Province', 'Khar Road', 'Faysh Khabur', 'Narjis', 'Hashim']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['States and territories established in the 11th century', 'History of the Rhineland', 'Former states and territories of Rhineland-Palatinate', 'History of the Westerwald']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Communes of PyrÃ©nÃ©es-Atlantiques', 'Populated places in Badakhshan Province', 'Irish-language feminine given names']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>Sayn</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['Sayn-Wittgenstein-Berleburg', 'Sayn-Wittgenstein-Hachenburg', 'Sayn-Wittgenstein-Hohenstein', 'Sayn-Wittgenstein-Karlsburg', 'Sayn-Wittgenstein-Ludwigsburg', 'Sayn-Wittgenstein-Sayn', 'Sayn-Wittgenstein-Sayn-Altenkirchen', 'Sayn-Wittgenstein-Vallendar', 'Sponheim-Sayn', 'Sayn-Homburg']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['States and territories established in the 11th century', 'History of the Rhineland', 'Former states and territories of Rhineland-Palatinate', 'History of the Westerwald']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Communes of PyrÃ©nÃ©es-Atlantiques', 'Populated places in Badakhshan Province', 'Irish-language feminine given names']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>List of highest domestic net collection of Hindi films</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['List of highest-grossing Indian films', '100 Crore Club', '1000 Crore Club', 'List of highest-grossing Indian films in overseas markets', 'List of highest-grossing films in India']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['Lists of Indian films']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Bollywood', 'Cinema of India', 'Lists of Indian films']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>List of highest domestic net collection of Hindi films</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['Bibliography of Hindi cinema', 'List of Bollywood films', 'List of highest-grossing Bollywood films', 'Hindi Cinema', 'Lists of Bollywood films']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['Lists of Indian films']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Bollywood', 'Cinema of India', 'Lists of Indian films']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>List of highest domestic net collection of Hindi films</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['List of highest-grossing Indian films', '100 Crore Club', '1000 Crore Club', 'List of highest-grossing Indian films in overseas markets', 'List of highest-grossing films in India']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['Lists of Indian films']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Bollywood', 'Cinema of India', 'Lists of Indian films']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>Circle</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['Apeirogon', 'Unit circle', 'Bankoff circle', 'Woo circles', 'Circle']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['Curves', 'Circles', 'Pi', 'Conic sections', 'Elementary shapes']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Elementary geometry', 'Circles', 'Spherical trigonometry']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>Circle</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['Apeirogon', 'Circle', 'Circumscribed circle', 'Triangle', 'Apeirogonal prism']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['Curves', 'Circles', 'Pi', 'Conic sections', 'Elementary shapes']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Elementary geometry', 'Circles', 'Spherical trigonometry']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>Circle</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>["Archimedes' twin circles", 'Affine sphere', 'Annulus (mathematics)', 'Sphere', 'Translation of axes', 'Unit circle', 'Apollonian circles', 'Chromatic circle', 'Ford circle', 'Circle of antisimilitude', 'Carlyle circle', 'Bankoff circle', 'Archimedean circle', 'Schoch circles', 'Woo circles', 'Nine-point circle', 'Excircle', 'Lester circle', 'Malfatti circles', 'Parry circle', 'Great circle', 'Villarceau circles']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['Curves', 'Circles', 'Pi', 'Conic sections', 'Elementary shapes']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Elementary geometry', 'Circles', 'Spherical trigonometry']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>Object 785</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['Leopard 2-140', 'Pz 87-140', 'Object 195', 'Type 99KM', 'Object 292']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['Abandoned military projects of the Soviet Union', 'Main battle tanks of the Cold War', 'Cold War tanks of the Soviet Union', 'Trial and research tanks of the Soviet Union', 'Main battle tanks of the Soviet Union']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Kosmos satellites', 'Supernova remnants', 'World War II submarines of Germany']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>Object 785</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['List of minor planets and comets visited by spacecraft', 'Interstellar comet', 'B612 Foundation', 'List of comets visited by spacecraft', '46610 BÃ\x83Â©sixdouze']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['Abandoned military projects of the Soviet Union', 'Main battle tanks of the Cold War', 'Cold War tanks of the Soviet Union', 'Trial and research tanks of the Soviet Union', 'Main battle tanks of the Soviet Union']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Kosmos satellites', 'Supernova remnants', 'World War II submarines of Germany']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>Object 785</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['CATTB', 'Leopard 2-140', 'Pz 87-140', 'Object 195', 'Type 99KM', 'Object 292']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['Abandoned military projects of the Soviet Union', 'Main battle tanks of the Cold War', 'Cold War tanks of the Soviet Union', 'Trial and research tanks of the Soviet Union', 'Main battle tanks of the Soviet Union']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Kosmos satellites', 'Supernova remnants', 'World War II submarines of Germany']</text>"""

In [8]:
HTML(o)